In [ ]:
# sample 0

import numpy as np
import math
import os
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from google.colab import drive
from torchsummary import summary

drive.mount('/content/gdrive')

class ConvBlock(nn.Module):
    def __init__(self, input_size, output_size, kernel_size=3, stride=1, padding=1, bias=True, activation='prelu', norm=None):
        super(ConvBlock, self).__init__()
        self.conv = torch.nn.Conv2d(input_size, output_size, kernel_size, stride, padding, bias=bias)

        self.norm = norm
        if self.norm =='batch':
            self.bn = torch.nn.BatchNorm2d(output_size)
        elif self.norm == 'instance':
            self.bn = torch.nn.InstanceNorm2d(output_size)
        
        # self.bn : Conv Layer 출력에서 normalization 을 Instance 로 할지 Batch 로 할지 선택
        
        self.activation = activation
        if self.activation == 'relu':
            self.act = torch.nn.ReLU(True)
        elif self.activation == 'prelu':
            self.act = torch.nn.PReLU()
        elif self.activation == 'lrelu':
            self.act = torch.nn.LeakyReLU(0.2, True)
        elif self.activation == 'tanh':
            self.act = torch.nn.Tanh()
        elif self.activation == 'sigmoid':
            self.act = torch.nn.Sigmoid()
        
        # self.act : Conv Layer 출력 Activation Function 선택
        
    def forward(self, x):
        if self.norm is not None:
            out = self.bn(self.conv(x))
        else:
            out = self.conv(x)

        if self.activation is not None:
            return self.act(out)
        else:
            return out

class SRCNN(nn.Module):
    def __init__(self, num_channels, base_filter, scale_factor):
        super(SRCNN, self).__init__()
        
        self.layer1 = ConvBlock(num_channels, base_filter, kernel_size=9, stride=1, padding=4, activation='relu', norm=None)
        self.layer2 = ConvBlock(base_filter, base_filter // 2, kernel_size=1, stride=1, padding=0, activation='relu', norm=None)   
        self.layer3 = ConvBlock(base_filter // 2, 3, kernel_size=5, stride=1, padding=2, activation=None, norm=None)
       
    def forward(self, x):
        f1 = self.layer1(x)
        f2 = self.layer2(f1)
        y = self.layer3(f2)
        
        return y

'''
1. GPU 설정 및 parameter print
'''
gpus_list = range(1)
cudnn.benchmark = True

cuda = True
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

model = SRCNN(num_channels = 16, base_filter=32, scale_factor=3)

'''
4. GPU 사용 여부 및 pretrained model 사용 여부 설정
'''
if cuda:
    model = model.cuda(gpus_list[0])
    model = torch.nn.DataParallel(model, device_ids=gpus_list)

summary(model, (16,32,32))
    

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]          41,504
              ReLU-2           [-1, 32, 32, 32]               0
         ConvBlock-3           [-1, 32, 32, 32]               0
            Conv2d-4           [-1, 16, 32, 32]             528
              ReLU-5           [-1, 16, 32, 32]               0
         ConvBlock-6           [-1, 16, 32, 32]               0
            Conv2d-7            [-1, 3, 32, 32]           1,203
         ConvBlock-8            [-1, 3, 32, 32]               0
             SRCNN-9            [-1, 3, 32, 32]               0
Total params: 43,235
Trainable params: 43,235
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.06
Forward

In [ ]:
# sample 1

import numpy as np
import math
import os
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from google.colab import drive
from torchsummary import summary

drive.mount('/content/gdrive')

class Channel_weighted_Conv(nn.Module):
  def __init__(self, input_size, kernel_size, stride=1, bias=True):
    super(Channel_weighted_Conv, self).__init__()
    self.input_size = input_size 
    # input_size = r^2

    self.conv = torch.nn.Conv2d(1, 1, kernel_size, stride, padding=int((kernel_size-1)/2), bias=bias)
    self.PS =  nn.PixelShuffle (upscale_factor = int(math.sqrt(input_size)))
    self.inner_conv = torch.nn.Conv2d(1, 1, kernel_size=int(math.sqrt(input_size)), stride=int(math.sqrt(input_size)), padding=0, bias=bias)

    '''
    apply 함수로 initialization 하기 힘들기 때문에 class 내에서 initialization 을 수행하도록 함
    '''
    for m in self.modules():
        # 이 class 에서 정의한 self.(변수)[인스턴스 변수] 목록 (conv, PS, inner_conv) 불러오기 
            classname = m.__class__.__name__
            # 불러온 module의 class 이름
            if classname.find('Conv2d') != -1:
                torch.nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    m.bias.data.zero_()
            # Conv2d Layer 라면 He 방법으로 해당 Layer 의 weight 들을 초기화, bias 가 True 면 bias 를 0 으로 초기화
            elif classname.find('ConvTranspose2d') != -1:
                torch.nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    m.bias.data.zero_()
           # ConvTranspose2d Layer 라면 He 방법으로 해당 Layer 의 weight 들을 초기화, bias 가 True 면 bias 를 0 으로 초기화:
             
            
            
  def forward(self, x):
    s = self.conv(x[:, 0:1, :, :])

    for i in range(1, self.input_size):
      m = self.conv(x[:, i:i+1, :, :])
      s = torch.cat([s, m], dim=1)
    
    ps = self.PS(s)
    out = self.inner_conv(ps)
    return out

class WCNN_block(nn.Module):
  def __init__(self, input_size, output_size, kernel_size):
    super(WCNN_block, self).__init__()
    self.filters = [0]*output_size

    '''
    여기서 filter 갯수(=output_size)만큼 Channel_weighted_Conv 을 list 형태로 담고,
    '''
    self.filters = nn.ModuleList([Channel_weighted_Conv(input_size, kernel_size, stride=1, bias=True) for _ in range(0, output_size)])
    self.filter_num = output_size
  
  def forward(self, x):
    '''
    여기서 각 Channel_weighted_Conv에 x(입력) 를 대입하고 나온 출력을 concatenation 해 준다.
    -> ModuleList 는 list 안의 각 Module 을 하나씩 접근 할 수 있게 해 줌
    '''
    out = self.filters[0](x)
    for i in range(1, self.filter_num):
      filtering = self.filters[i](x)
      out = torch.cat([out, filtering], dim=1)
    # out.shape = [batch, K, N, N]
    # out = 각 filter 들의 WCNN 결과의 concatenation
    return out

class WCNN(nn.Module):
    def __init__(self):
        super(WCNN, self).__init__()
        
        '''
        1. layer2 만을 WCNN_block 으로 바꿈
        2. WCNN_block 의 input_size = r^2 이어야 하는 것을 유의함
        '''
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,64,kernel_size=9),
            nn.ReLU(),
        )
        self.layer2 = nn.Sequential(
            WCNN_block(input_size=64, output_size=32, kernel_size=3),
            nn.ReLU(),
        )
        # WCNN_block 의 Kernel_size = 홀수 여야 출력이미지의 size 가 입력과 같아진다
        self.layer3 = nn.Sequential(
            nn.Conv2d(32,3,kernel_size=5),
        )

    def forward(self, x):
        f1 = self.layer1(x)
        f2 = self.layer2(f1)
        y = self.layer3(f2)
        
        return y

'''
1. GPU 설정 및 parameter print
'''
gpus_list = range(1)
cudnn.benchmark = True

cuda = True
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

model = WCNN()

'''
4. GPU 사용 여부 및 pretrained model 사용 여부 설정
'''
if cuda:
    model = model.cuda(gpus_list[0])
    model = torch.nn.DataParallel(model, device_ids=gpus_list)

# summary(model, (3,32,32))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check


In [ ]:
# sample 2

import numpy as np
import math
import os
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from google.colab import drive
from torchsummary import summary

drive.mount('/content/gdrive')

class Channel_weighted_Conv(nn.Module):
  def __init__(self, input_size, kernel_size, stride=1, bias=True):
    super(Channel_weighted_Conv, self).__init__()
    self.input_size = input_size 
    # input_size = r^2

    self.conv = torch.nn.Conv2d(1, 1, kernel_size, stride, padding=int((kernel_size-1)/2), bias=bias)
    self.PS =  nn.PixelShuffle (upscale_factor = int(math.sqrt(input_size)))
    self.inner_conv = torch.nn.Conv2d(1, 1, kernel_size=int(math.sqrt(input_size)), stride=int(math.sqrt(input_size)), padding=0, bias=bias)

  def forward(self, x):
    s = self.conv(x[:, 0:1, :, :])

    for i in range(1, self.input_size):
      m = self.conv(x[:, i:i+1, :, :])
      s = torch.cat([s, m], dim=1)
    
    print('S의 shape = ',s.shape)
    ps = self.PS(s)
    print('PS의 shape = ',ps.shape)
    out = self.inner_conv(ps)
    print('OUT의 shape = ',out.shape)
    return out
    

'''
1. GPU 설정 및 parameter print
'''
gpus_list = range(1)
cudnn.benchmark = True

cuda = True
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

model = Channel_weighted_Conv(input_size=16, kernel_size=5)

'''
4. GPU 사용 여부 및 pretrained model 사용 여부 설정
'''
if cuda:
    model = model.cuda(gpus_list[0])
    model = torch.nn.DataParallel(model, device_ids=gpus_list)
    
summary(model, (16,32,32))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
S의 shape =  torch.Size([2, 16, 32, 32])
PS의 shape =  torch.Size([2, 1, 128, 128])
OUT의 shape =  torch.Size([2, 1, 32, 32])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 1, 32, 32]              26
            Conv2d-2            [-1, 1, 32, 32]              26
            Conv2d-3            [-1, 1, 32, 32]              26
            Conv2d-4            [-1, 1, 32, 32]              26
            Conv2d-5            [-1, 1, 32, 32]              26
            Conv2d-6            [-1, 1, 32, 32]              26
            Conv2d-7            [-1, 1, 32, 32]              26
            Conv2d-8            [-1, 1, 32, 32]              26
            Conv2d-9            [-1, 1, 32, 32]              26
           Conv2d-10            [-1, 1, 3

In [ ]:
# sample 3

import numpy as np
import math
import os
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from google.colab import drive
from torchsummary import summary

drive.mount('/content/gdrive')

class test(nn.Module):
  def __init__(self, input_size):
    super(test, self).__init__()
    self.input_size = input_size
    self.conv = torch.nn.Conv2d(1, 1, 3, 1, 1, bias=True)

  def forward(self, x):
    out = self.conv(x[:, 0:1, :, :])

    for i in range(1, self.input_size):
      m = self.conv(x[:,i:i+1,:,:])
      out = torch.cat([out, m], dim=1)
    return out
  
  def num_dim(self, x):
    size = x.size() # 입력의 모든 차원
    print(size)
    

'''
1. GPU 설정 및 parameter print
'''
gpus_list = range(1)
cudnn.benchmark = True

cuda = True
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

model = test(input_size=16)

'''
4. GPU 사용 여부 및 pretrained model 사용 여부 설정
'''
if cuda:
    model = model.cuda(gpus_list[0])
    model = torch.nn.DataParallel(model, device_ids=gpus_list)
    
summary(model, (16,32,32))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 1, 32, 32]              10
            Conv2d-2            [-1, 1, 32, 32]              10
            Conv2d-3            [-1, 1, 32, 32]              10
            Conv2d-4            [-1, 1, 32, 32]              10
            Conv2d-5            [-1, 1, 32, 32]              10
            Conv2d-6            [-1, 1, 32, 32]              10
            Conv2d-7            [-1, 1, 32, 32]              10
            Conv2d-8            [-1, 1, 32, 32]              10
            Conv2d-9            [-1, 1, 32, 32]              10
           Conv2d-10            [-1, 1, 32, 32]              10
           Conv2d-11            [-1, 1, 32, 32]              10
           Conv2d-12            [-1,

In [ ]:
# sample 4

import torch

t1 = torch.tensor([[[[1,2,3],
                   [4,5,6],
                   [7,8,9]],
                  [[1,2,3],
                   [4,5,6],
                   [7,8,9]]]
                 ,[[[1,2,3],
                   [4,5,6],
                   [7,8,9.]],
                  [[1,2,3],
                   [4,5,6],
                   [7,8,9]]]])

t2 = torch.tensor([[[[0,0,0],
                   [4,5,6],
                   [7,8,9]],
                  [[0,0,0],
                   [4,5,6],
                   [7,8,9]]]
                 ,[[[0,0,0],
                   [4,5,6],
                   [7,8,9.]],
                  [[0,0,0],
                   [4,5,6],
                   [7,8,9]]]])

print(t1.shape)
print(t2.shape)
print(torch.cat([out, t2], dim=1).shape)
print(t1[:, 1:2,:,:].size())

def printf(a):
  print(a)
  return 0



torch.Size([2, 2, 3, 3])
torch.Size([2, 2, 3, 3])


NameError: ignored

In [ ]:
def get_patch(img_in, img_tar, img_bic, patch_size, scale, ix=-1, iy=-1):
    (ih, iw) = img_in.size
    (th, tw) = (scale * ih, scale * iw)

    patch_mult = scale #if len(scale) > 1 else 1
    tp = patch_mult * patch_size
    # tp = upscale_factor*n (n = patch_size)
    ip = tp // scale
    # ip = n (n = patch_size)
    
    if ix == -1:
        ix = random.randrange(0, iw - ip + 1)
    if iy == -1:
        iy = random.randrange(0, ih - ip + 1)

    (tx, ty) = (scale * ix, scale * iy)

    img_in = img_in.crop((iy,ix,iy + ip, ix + ip))
    # 원본이미지 임의의 영역을 n x n 크기로 자름
    img_tar = img_tar.crop((ty,tx,ty + tp, tx + tp))
    # downscale 한 이미지 임의의 영역을 upscale_factor*n x upscale_factor*n 크기로 자름
    img_bic = img_bic.crop((ty,tx,ty + tp, tx + tp))
    # upscale 한 이미지 임의의 영역을 upscale_factor*n x upscale_factor*n 크기로 자름            
    info_patch = {
        'ix': ix, 'iy': iy, 'ip': ip, 'tx': tx, 'ty': ty, 'tp': tp}
    # info_patch 에는 어떤 크기로 crop 했는지에 대한 정보가 있음
    return img_in, img_tar, img_bic, info_patch

'''
입력 이미지의 가로/세로 pixel 수가 upscale_factor 의 배수가 아니면 입력 이미지를 crop 하여 가로 세로를 upscale_factor 의 배수로 맞춰주는 함수를
'''
def get_patch()


    
'''
8.사용자 정의 data 로드 - Test Data 
   -> file 이름까지 사용자 정의 데이터화 시킴
   -> get_patch 함수를 사용하지 않기 때문에, SRCNN 의 입력으로 (원본 이미지 xscale) 된 이미지가 들어감
'''
class DatasetFromFolderEval(data.Dataset):
    def __init__(self, lr_dir, upscale_factor, transform=None):
        super(DatasetFromFolderEval, self).__init__()
        self.image_filenames = [join(lr_dir, x) for x in listdir(lr_dir) if is_image_file(x)]
        self.upscale_factor = upscale_factor
        self.transform = transform

    def __getitem__(self, index):
        target = load_img(self.image_filenames[index])
        # target = load_img 함수를 이용하여 index 번째 image 를 RGB 로 불러온 이미지
        _, file = os.path.split(self.image_filenames[index])
        # file = 이미지 경로 list 에서 file 이름만 잘라 냄

        input = target.resize((int(target.size[0]/self.upscale_factor),int(target.size[1]/self.upscale_factor)), Image.BICUBIC) 
        # input = target 으로 불러온 이미지를 Bicubic 으로 downscale 한 이미지
        bicubic = rescale_img(input, self.upscale_factor)
        # bicubic = rescale_img 함수로 input 을 다시 같은 비율로 upscale 한 이미지

        input, target, bicubic, _ = get_patch(input,target,bicubic,self.patch_size, self.upscale_factor)
        # input, target, bicubic 이미지를 (upscale_factor*patch_size x upscale_factor*patch_size) 로 crop
        
        if self.transform:
            input = self.transform(input)
            bicubic = self.transform(bicubic)
            target = self.transform(target)
         # self.transform = True 라면, 세 이미지를 PIL image 에서 Tensor로 바꾸어 줌
        
        return input, bicubic, target, file
        # file 이름까지 return
      
    def __len__(self):
        return len(self.image_filenames)